# CNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

basicpath = '/content/drive/MyDrive/LG_DIC_lecture/2일차/실습'


In [ ]:
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import os

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
path = os.path.join(basicpath, 'Dataset/Classification/')

## CIFAR Image

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/cifar10.png')

## CIFAR data import

#### OFFline pickle data

In [ ]:
# def unpickle(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict
# data = unpickle(os.path.join(os.path.join(path, 'cifar-10-batches-py') ,'/data_batch_1'))

#### Online PIL image data

In [ ]:
# # For online situation
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)

#### OFFline PIL data

In [ ]:
# OFFline PIL data
trainset = torchvision.datasets.CIFAR10(root=path, train=True, 
                                        download=False)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
for i in range(5):
    index = np.random.randint(len(trainset), dtype=int)
    image, label = trainset[index]
    image = image.resize((128, 128))
    display(image)
    print(classes[label])

### Transform PIL data to torch data and normalization

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                        download=False, transform=transform)


In [ ]:
a, _ =trainset[0]
a

### Make Dataloader

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

## Learning with CNN - Convolutional Neural Network

#### CNN: Convolution - (padding) - ReLU - Pooling layer로 이루어진 일련의 layer가 포함된 Network  
     인접 픽셀간의 상관관계를 고려한 Network 형태

### Convolution layer

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/convolution.png')

### Padding layer

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/padding.png')

### Pooling layer

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/pooling.png')

## CNN model

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/CNN.png')

In [ ]:
from IPython.display import Image
Image(basicpath + 'Image/CNN_layers.png')

## Define Model 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

In [ ]:
def conv_block(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 5), 
        nn.ReLU(), 
        nn.MaxPool2d(2, 2)
    )
    return model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature = nn.Sequential(
            conv_block(3, 6), 
            conv_block(6, 16)
        )
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
        
#         x = self.conv1(x)
#         x = self.pool(F.relu(self.conv2(x)))
        x = self.feature(x)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)


## Learning the model

In [ ]:
learning_rate = 0.001

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

In [ ]:
epochs = 20

In [ ]:
for epoch in range(epochs):
    running_cost = 0.0

    for step, (batch_data) in enumerate(trainloader):
        batch_x, batch_y = batch_data[0].to(device), batch_data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = net(batch_x)
        cost = criterion(outputs, batch_y)

        cost.backward()
        optimizer.step()
        
        running_cost += cost.item()
        if step % 2000 == 1999:
            print('[%d, %5d] cost: %.3f' % (epoch + 1, step + 1, running_cost / 2000))
            running_cost = 0.0
            

## 정확도 판단

#### Test dataset import

In [ ]:
testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                        download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset),
                                          shuffle=False, num_workers=2)

### Confusion matrix and scores

In [ ]:
test_iter = iter(testloader)
test_x, test_labels = test_iter.next()

In [ ]:
outputs = net(test_x.to(device))
_, predicted = torch.max(outputs, 1)

In [ ]:
predicted

#### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
predicted = predicted.cpu()
print(confusion_matrix(test_labels, predicted))

#### Precision

In [ ]:
from sklearn.metrics import precision_score
print(precision_score(test_labels, predicted, average=None))
print(precision_score(test_labels, predicted, average='weighted'))

#### Recall

In [ ]:
from sklearn.metrics import recall_score
print(recall_score(test_labels, predicted, average=None))
print(recall_score(test_labels, predicted, average='weighted'))